# Scraping for Putin's close associates
https://www.spisok-putina.org/

The website (https://www.spisok-putina.org/) **“PUTIN’S LIST”** is created by the Free Russia Forum in order **to search, consolidate and unify data** on those responsible for creating and running a system of unchecked power and suppressing civil liberties both in Russia and abroad.

The project aims to gather information related to persons of the following categories:

* The **“POWER-HOLDERS”** of Russia, who seized and unlawfully hold power in the country;
* The **“EXECUTORS”**, who, as public servants and MPs, directly implement the Kremlin’s policy of felonious retention of power in the country;
* The **“TRANSGRESSORS”**, from among the judges, prosecutors, investigators, lawyers, law enforcement officers and their agents responsible for the direct implementation of the repression and prosecution of the opposition;
* The **“AGGRESSORS”**, who are responsible for carrying out the policies of war, occupation and chaos against other countries, in violation of international law, in the political interests of the leadership of the country;
* The **“BENEFICIARIES”**, foreign partners of the Russian government who contribute to the implementation of its felonious plans to undermine international law and global peace, justice and democracy.
* The **“OLIGARCHS AND CORRUPT OFFICIALS”**, who are responsible for plundering Russia on a state-wide scale, by direct membership in the Russian government or proximity to it through relatives and other proxies;
* The **“PROPAGANDISTS”**, who are responsible for creating and disseminating state information, deliberately deceiving people in the interests of the political leadership and creating an atmosphere of widespread hatred and intolerance;
* The **“ACCOMPLICES”**, who reap personal profit from cooperation with government authorities, to the detriment of the interests of the country and its citizens;

In [7]:
from bs4 import BeautifulSoup as bs
import requests
import urllib.request
import re
import pandas as pd
from transliterate import translit

I started by creating a list that only keeps the first names of individuals. I used this for adding the name to the original url 
so I can access and scrape each page apart

In [8]:
base_url = "https://www.spisok-putina.org/en/personas/page/#/"

"""this list keeps only the first name"""
persons_list = []

for page_number in range(1, 141): # there are 140 webpages for each particular individual
    url = base_url.replace("#", str(page_number))
    response = requests.get(url) # send request
    soup = bs(response.content, 'html.parser')  # bs object
    person_names = soup.find_all('div', class_='preview-material__content') # extract all person names

    # append the first word of each name to the persons_list
    for person_name in person_names:
        full_name = person_name.text.strip().replace('\n', ' ').replace('  ', ' ')
        first_word = full_name.split()[0]
        persons_list.append(first_word.lower())

In [9]:
# example
persons_list[:10]

['abakarov',
 'abdullazyanov',
 'abramov',
 'abramovich',
 'afanasieva',
 'afanasov',
 'afanasyeva',
 'afasizhev',
 'afonin',
 'agabekyan']

I will add the name of each individual to the base url, than I start to acces each page and scrape for needed information like name, citizenship, date of birth, category, biography. Finally, create a dataframe with that information.

In [10]:
base_url = "https://www.spisok-putina.org/en/personas/" # this is the base url

data = [] # for df creation

In [12]:
for name in persons_list:
    url = base_url + name
    try:
        page = requests.get(url)
        page.raise_for_status()  # Raise an HTTPError for bad responses
        soup = bs(page.content, 'html.parser')

        # Extracting information from the webpage
        name_element = soup.select_one('.material__title')
        name = name_element.text.strip() if name_element else ''

        dob_element = soup.find('strong', text="Date of Birth:")
        date_of_birth = dob_element.find_next('br').next_sibling.strip() if dob_element else ''

        citizenship_element = soup.find('strong', text="Citizenship:")
        citizenship = citizenship_element.find_next('br').next_sibling.strip() if citizenship_element else ''

        categories_element = soup.find('strong', text="Categories:")
        categories = ""
        if categories_element:
            categories_tags = categories_element.find_next_siblings('a')
            categories_list = [tag.text.strip() for tag in categories_tags]
            categories = ', '.join(categories_list)

        biography_element = soup.find('div', class_='entry-content')
        for unwanted_section in biography_element.select('.section-accused, .section-materials'):
            unwanted_section.extract()

        biography_paragraph = biography_element.find('p')
        biography = biography_paragraph.text.strip() if biography_paragraph else ''

        data.append({
            'Name': name,
            'Date of Birth': date_of_birth,
            'Citizenship': citizenship,
            'Categories': categories,
            'Biography': biography
        })
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {name}: {e}")

C:\Users\Asus\AppData\Local\Temp\ipykernel_752\3730135327.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  dob_element = soup.find('strong', text="Date of Birth:")
C:\Users\Asus\AppData\Local\Temp\ipykernel_752\3730135327.py:15: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  citizenship_element = soup.find('strong', text="Citizenship:")
C:\Users\Asus\AppData\Local\Temp\ipykernel_752\3730135327.py:18: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  categories_element = soup.find('strong', text="Categories:")


Error fetching data for timchenko: 404 Client Error: Not Found for url: https://www.spisok-putina.org/personas/timchenko/
Error fetching data for timchenko: 404 Client Error: Not Found for url: https://www.spisok-putina.org/personas/timchenko/
Error fetching data for yevtushenkov: 404 Client Error: Not Found for url: https://www.spisok-putina.org/en/personas/yevtushenkov
Error fetching data for корнев: 404 Client Error: Not Found for url: https://www.spisok-putina.org/en/personas/%D0%BA%D0%BE%D1%80%D0%BD%D0%B5%D0%B2


In [19]:
# example
data[0]

{'Name': 'Abakarov Khizri  Magomedovich',
 'Date of Birth': '28 June 1960',
 'Citizenship': 'Russia',
 'Categories': 'Beneficiaries',
 'Biography': 'Russian statesman and public figure, entrepreneur. Head of the administration of the city of Derbent of the Republic of Dagestan. State Secretary of the Republic of Dagestan since November 10, 2020. Member of the State Duma of the Russian Federation since 2021 from the United Russia party'}

In [13]:
mydata = pd.DataFrame(data)

In [20]:
mydata.head()

,Name,Date of Birth,Citizenship,Categories,Biography
0,Abakarov Khizri Magomedovich,28 June 1960,Russia,Beneficiaries,"Russian statesman and public figure, entrepren..."
1,Abdullazyanov Edward Yunusovich,8 April 1957,Russia,Accomplices,Rector of Kazan State Power Engineering Univer...
2,Abramov Ivan Nikolaevich,16 June 1978,Russia,Executors,Deputy Chairman of the Federation Council Comm...
3,Абрамович Роман Аркадьевич,,,,Роман Абрамович родился 24 октября 1966 года в...
4,Afanasieva (Berg) Yulia,24 February 1988,Russia,Executors,An employee of the African Association for Fre...


**There are some inaccuracies in data like for example some information are in russian, because there are duplicates (persons who share the same name) so his particular webpage is different. And, in the case of duplicates, I have scraped the russian version of webpage. So I will do some data transformation and preprocessing for adjusting these errors and gather as much information as I can.**

In [21]:
list_ = mydata['Name'].tolist()

#### Transforming ru chars names in English

In [22]:
english_first_words = []

for name in list_:
    name_parts = name.split() # split the name into words
    if len(name_parts) > 0: # check if the name has at least one word
        first_word = name_parts[0]
        if any(ord(char) > 128 for char in first_word): # check if the first word it's in Russian (if contains any non ASCII characters)
            english_first_word = translit(first_word, 'ru', reversed=True) # translate the first word from russian to en
            english_first_words.append(english_first_word.lower())

Some persons share the same name like for example the name "Ivanov". Then, the webpage coresponding to a particular person will be:
https://www.spisok-putina.org/personas/ivanov/, or https://www.spisok-putina.org/personas/ivanov-2/, or https://www.spisok-putina.org/personas/ivanov-3/ and so on. So I created a function to ad "-2" or "-3" (coressponding number), based on how many occurencies of that name are in the above list.

In [24]:
def transform_items(lst):
    item_count = {}
    transformed_list = []

    for item in lst:
        if item in item_count:
            item_count[item] += 1
        else:
            item_count[item] = 1

        count = item_count[item]

        if count == 1:
            transformed_list.append(f"{item}-2")
        else:
            transformed_list.append(f"{item}-{count}")

    return transformed_list

In [25]:
newlist_ = transform_items(english_first_words)

**Now, I can acces that pages too, saving a lot of information.**

In [27]:
newlist_[:10]

['abramovich-2',
 'afasizhev-2',
 'agalarov-2',
 'agramunt-2',
 'akimov-2',
 'akimov-2',
 'aksenov-2',
 'alekperov-2',
 'ammosov-2',
 'ananskih-2']

Here I just used the same script as above

In [29]:
data2 = []

for name in newlist_:
    url = base_url + name
    try:
        page = requests.get(url)
        soup = bs(page.content, 'html.parser')
        name_element = soup.select_one('.material__title')
        name = name_element.text.strip() if name_element else ''
        #print("Name:", name)

        dob_element = soup.find('strong', text="Date of Birth:")
        date_of_birth = dob_element.find_next('br').next_sibling.strip() if dob_element else ''
        #print("Date of Birth:", date_of_birth)

        citizenship_element = soup.find('strong', text="Citizenship:")
        citizenship = citizenship_element.find_next('br').next_sibling.strip() if citizenship_element else ''
        #print("Citizenship:", citizenship)

        categories_element = soup.find('strong', text="Categories:")
        categories = ""
        if categories_element:
            categories_tags = categories_element.find_next_siblings('a')
            categories_list = [tag.text.strip() for tag in categories_tags]
            categories = ', '.join(categories_list)

        #print("Categories:", categories)

        biography_element = soup.find('div', class_='entry-content')
        if biography_element:
            for unwanted_section in biography_element.select('.section-accused, .section-materials'):
                unwanted_section.extract()

            biography_paragraph = biography_element.find('p')
            biography = biography_paragraph.text.strip() if biography_paragraph else ''
            #print("Biography:", biography)

            data2.append({
                'Name': name,
                'Date of Birth': date_of_birth,
                'Citizenship': citizenship,
                'Categories': categories,
                'Biography': biography
            })
        else:
            pass
    except requests.exceptions.RequestException:
        pass

df2 = pd.DataFrame(data2)

C:\Users\Asus\AppData\Local\Temp\ipykernel_752\2645672818.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  dob_element = soup.find('strong', text="Date of Birth:")
C:\Users\Asus\AppData\Local\Temp\ipykernel_752\2645672818.py:16: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  citizenship_element = soup.find('strong', text="Citizenship:")
C:\Users\Asus\AppData\Local\Temp\ipykernel_752\2645672818.py:20: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  categories_element = soup.find('strong', text="Categories:")


Now I have to remove the instances from dataframe that are in Russian and than concatanate it with df2, which is bassically the english translation of that deleted instances.

In [30]:
import unicodedata # this is for check if text contains ru characters

In [31]:
# check if text contains ru characters
def contains_russian(text):
    for char in text:
        if 'CYRILLIC' in unicodedata.name(char, ''):
            return True
    return False


# remove rows from the df that contain ru chars
def remove_rows_with_russian(df):
    indices_to_remove = []

    for index, row in df.iterrows():
        if any(contains_russian(str(cell)) for cell in row):
            indices_to_remove.append(index)

    cleaned_df = df.drop(indices_to_remove)
    return cleaned_df

In [32]:
without_ru = remove_rows_with_russian(mydata)

## Full dataset

In [33]:
full_dataset = pd.concat([without_ru, df2], ignore_index = True)

In [34]:
full_dataset.head()

,Name,Date of Birth,Citizenship,Categories,Biography
0,Abakarov Khizri Magomedovich,28 June 1960,Russia,Beneficiaries,"Russian statesman and public figure, entrepren..."
1,Abdullazyanov Edward Yunusovich,8 April 1957,Russia,Accomplices,Rector of Kazan State Power Engineering Univer...
2,Abramov Ivan Nikolaevich,16 June 1978,Russia,Executors,Deputy Chairman of the Federation Council Comm...
3,Afanasieva (Berg) Yulia,24 February 1988,Russia,Executors,An employee of the African Association for Fre...
4,Afanasov Mikhail Alexandrovich,15 July 1953,Russia,Executors,Member of the Committee of the Federation Coun...


In [35]:
full_dataset.describe().transpose()

,count,unique,top,freq
Name,1614,1437,Orlov Aleksey Maratovich,5
Date of Birth,1614,1171,,263
Citizenship,1614,17,Russia,1437
Categories,1614,27,Executors,732
Biography,1614,1084,Deputy of the State Duma of the 8th convocatio...,100


In [36]:
full_dataset['Citizenship'].value_counts().head(3)

Russia     1437
            143
Ukraine       9
Name: Citizenship, dtype: int64

In [37]:
full_dataset['Categories'].value_counts().head(8)

Executors                          732
Accomplices                        389
Propagandists                      184
Law-destroyers                      81
Oligarchs and corrupt officials     50
                                    42
Beneficiaries                       38
Executors, Law-destroyers           24
Name: Categories, dtype: int64